### Power simulation

Test of comparative statics

In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from itertools import product 
from scipy.stats import mannwhitneyu 

In [2]:
def compute_test_result(n_matching_grp_per_treatment, 
                        n_obs_per_matching_grp,
                        small_coal_worth, 
                        unit, 
                        err_type, 
                        err_scale, 
                        ind_set, 
                        test,
                        use_treatment_dummies, 
                        cov_type):
    """
    Run a statistical test on data generated by assumed DGP

    Parameters: 
    - experiment set-up
        * n_matching_grp_per_treatment # number of matching groups per treatment/session
        * n_obs_per_matching_grp # number of observations per matching group
        * small_coal_worth # list of treatment values
    - DGP:
        * err_type = "uniform" # choose: "uniform", "norm_corr"
        * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
        * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
    - specification:
        * unit # choose: "group", "matching_group"
        * use_treatment_dummies # choose: 0,1
        * test # choose: "reg", "mann_whitney" (one-sided)
        * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3" (if using group as a unit)
    
    Returns:
    - p value(s) 
    """
    # Generate independent variables
    
    n_treat = len(small_coal_worth)
    n_obs = n_obs_per_matching_grp * n_matching_grp_per_treatment * n_treat # total number of group observations
    
    Xval = np.array(sum([n_obs_per_matching_grp * n_matching_grp_per_treatment * [val] for val in small_coal_worth], [])) 
    X = pd.get_dummies(Xval, drop_first= True).values.astype(int) if use_treatment_dummies else Xval 
    X = sm.add_constant(X)

    if unit == "matching_group" or (unit == "group" and cov_type == "cluster"): 
        clusters = np.array(sum([n_obs_per_matching_grp * [cluster] for cluster in range(0, n_matching_grp_per_treatment * n_treat)], [])) 
        X = np.column_stack((X, clusters))

    # Assumed DGP 

    if err_type == "uniform":
        e = np.random.random(n_obs) * 2 * err_scale - err_scale
    elif err_type == "norm_corr": # generate correlated errors within each matching group
        cov = err_scale * np.random.random(size=(n_obs_per_matching_grp,n_obs_per_matching_grp))
        cov = cov + cov.T # make it symmetric 
        cov = np.dot(cov, cov.T) # make it positive semidefinite
        e = multivariate_normal.rvs(mean = np.zeros(n_obs_per_matching_grp), cov = cov, size = n_matching_grp_per_treatment * n_treat).flatten()
    e = e.round() # as subjects can only choose integer values

    es = np.ones(n_obs) * (100//3)  
    sh = (100 + Xval)//3  
    no_coord = np.zeros(n_obs)
    ind = {0: es, 1: sh, 2: no_coord} 

    rand = [np.random.choice(ind_set) for j in range(n_obs)] 
    y = np.array([ind[rand[i]][i] for i in range(n_obs)]) + np.array([0 if rand[i]==2 else e[i] for i in range(n_obs)])
    y = np.maximum(y,0) 
    y = np.minimum(y,100)

    if unit == "matching_group":
        y = np.array([np.mean(y[X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])
        X = np.array([np.mean(X[:,0:-1][X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])

    # Run statistical test (only on outcomes with successful coordination)

    y_reg = y[y>0] 
    X_reg = X[y>0]

    if test == "reg":
        if unit == "matching_group":
            results = sm.regression.linear_model.OLS(y_reg, X_reg).fit() 
        elif unit == "group":
            if cov_type == "cluster":
                results = sm.regression.linear_model.OLS(y_reg, X_reg[:, :-1]).fit(cov_type = cov_type, cov_kwds= {"groups": X_reg[:, -1]}) # clustered standard errors
            else:
                results = sm.regression.linear_model.OLS(y_reg, X_reg).fit(cov_type = cov_type) # robust standard errors

        return (results.pvalues[1], results.pvalues[2]) if use_treatment_dummies else results.pvalues[1]

    elif test == "mann_whitney": 
        # note that 'power_coeff1' and 'power_coeff2' take on a different meaning here than for regression
        results_1 = mannwhitneyu(y_reg[X_reg[:,1] == small_coal_worth[0]], y_reg[X_reg[:,1] == small_coal_worth[1]], alternative = "less", method = "auto")
        results_2 = mannwhitneyu(y_reg[X_reg[:,1] == small_coal_worth[0]], y_reg[X_reg[:,1] == small_coal_worth[2]], alternative = "less", method = "auto")
        
        return (results_1.pvalue, results_2.pvalue)

In [3]:
def power_simulation(num_runs, 
                     sig_level, 
                     n_matching_grp_per_treatment, 
                     n_obs_per_matching_grp, 
                     small_coal_worth, 
                     unit, 
                     err_type, 
                     err_scale,
                     ind_set, 
                     test,
                     use_treatment_dummies, 
                     cov_type):
    """
        Run a power simulation for our bargaining experiment 
    
        Parameters: 
        - simulation:
            * num_runs 
            * sig_level 
        - experiment set-up
            * n_matching_grp_per_treatment # number of matching groups per treatment/session
            * n_obs_per_matching_grp # number of observations per matching group
            * small_coal_worth # list of treatment values
        - DGP:
            * err_type = "uniform" # choose: "uniform", "norm_corr"
            * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
            * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
        - specification:
            * unit # choose: "group", "matching_group"
            * test # choose: "reg", "mann_whitney" (one-sided)
            * use_treatment_dummies # choose: 0,1
            * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3"
        
        Returns: 
        - Dataframe with p value(s) for each simulation run.
    """
    power_sim_results = pd.DataFrame({'p_value_1': np.zeros(num_runs), 'p_value_2': np.zeros(num_runs)}) if (use_treatment_dummies or test == "mann_whitney") else pd.DataFrame({'p_value_1': np.zeros(num_runs)}) 

    for run in range(num_runs):
        power_sim_results.loc[run, :] = compute_test_result(n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit, err_type, err_scale, ind_set, test, use_treatment_dummies, cov_type)

    power_sim_summary = dict([(key,value) for key, value in locals().items() if key not in ["power_sim_results", "run"]])

    p_reject_1 = np.mean(power_sim_results['p_value_1'] < sig_level)
    if use_treatment_dummies or test == "mann_whitney": 
        p_reject_2 = np.mean(power_sim_results['p_value_2'] < sig_level)
        
    power_sim_summary["power_coeff1"] = p_reject_1 
    power_sim_summary["power_coeff2"] = p_reject_2 if (use_treatment_dummies or test == "mann_whitney") else np.nan

    return power_sim_summary


In [4]:
# run power simulation over a set of plausible parameters

# parameters: simulation, DGP
num_runs = [5000]
sig_level = [0.05]
n_matching_grp_per_treatment = [5]
n_obs_per_matching_grp = [10]
small_coal_worth = [[10,30,90]]
err_type = ["uniform", "norm_corr"] 
err_scale =  [1,2,3]
ind_set = [[0,0,0,1,1,1,2], [0,1,1,1,1,2]]

# parameters: test specification 
# (use subset of a grid, as not all combinations make sense)
params = {  
            0: {"test": "reg", "unit": "group", "use_treatment_dummies": 1, "cov_type": "cluster"},
            1: {"test": "reg", "unit": "group", "use_treatment_dummies": 1, "cov_type": "HC3"},
            2: {"test": "reg", "unit": "group", "use_treatment_dummies": 0, "cov_type": "cluster"},
            3: {"test": "reg", "unit": "group", "use_treatment_dummies": 0, "cov_type": "HC3"},
            4: {"test": "mann_whitney", "unit": "group", "use_treatment_dummies": 0, "cov_type": np.nan},
            5: {"test": "reg", "unit": "matching_group", "use_treatment_dummies": 1, "cov_type": np.nan},
            6: {"test": "mann_whitney", "unit": "matching_group", "use_treatment_dummies": 0, "cov_type": np.nan},
            7: {"test": "reg", "unit": "matching_group", "use_treatment_dummies": 0, "cov_type": np.nan}
         }

# simulation
res = pd.DataFrame()

combinations = list(product(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, err_type, err_scale, ind_set, params))
for comb in combinations: 
    #print(comb)
    
    ps = power_simulation(   
            num_runs = comb[0], 
            sig_level = comb[1], 
            n_matching_grp_per_treatment = comb[2], 
            n_obs_per_matching_grp = comb[3], 
            small_coal_worth = comb[4], 
            unit = params[comb[8]]["unit"], 
            err_type = comb[5], 
            err_scale = comb[6],
            ind_set = comb[7], 
            test = params[comb[8]]["test"],
            use_treatment_dummies = params[comb[8]]["use_treatment_dummies"], 
            cov_type = params[comb[8]]["cov_type"]
        )
    r = pd.DataFrame.from_dict(ps, orient = "index").transpose()
    res = pd.concat([res, r], axis = 0)
    

Basic analysis/comparison of results
* cov_type: cluster vs HC3
* unit: group vs matching group
* test: parametric vs nonparametric

In [5]:
# make them hashable
res["small_coal_worth"] = res["small_coal_worth"].apply(lambda x: tuple(x))
res["ind_set"] = res["ind_set"].apply(lambda x: tuple(x))

In [6]:
res_pivot = res.pivot(index = ["err_type", "err_scale", "ind_set"], columns = ["test", "unit", "cov_type", "use_treatment_dummies"], values = ["power_coeff1", "power_coeff2"])
res_pivot

power_coeff1                  \
test                                               reg                   
unit                                             group                   
cov_type                                       cluster     HC3 cluster   
use_treatment_dummies                                1       1       0   
err_type  err_scale ind_set                                              
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)        0.446   0.742   0.985   
                    (0, 1, 1, 1, 1, 2)          0.7822  0.9544     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)       0.2102   0.597  0.8496   
                    (0, 1, 1, 1, 1, 2)          0.3566  0.7526  0.9962   
          3         (0, 0, 0, 1, 1, 1, 2)       0.1538   0.554  0.6264   
                    (0, 1, 1, 1, 1, 2)          0.2326  0.6438   0.933   
uniform   1         (0, 0, 0, 1, 1, 1, 2)       0.9918  0.9936  0.9992   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)       0.9802  0.9872  0.9996   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          3         (0, 0, 0, 1, 1, 1, 2)       0.9742  0.9802  0.9996   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   

                                                                               \
test                                              mann_whitney            reg   
unit                                                     group matching_group   
cov_type                                      HC3          NaN            NaN   
use_treatment_dummies                           0            0              1   
err_type  err_scale ind_set                                                     
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)  0.9956         0.72          0.088   
                    (0, 1, 1, 1, 1, 2)        1.0       0.9778         0.1644   
          2         (0, 0, 0, 1, 1, 1, 2)  0.9534       0.5752         0.0656   
                    (0, 1, 1, 1, 1, 2)        1.0         0.77         0.1136   
          3         (0, 0, 0, 1, 1, 1, 2)  0.8748       0.4894         0.0646   
                    (0, 1, 1, 1, 1, 2)     0.9942       0.6138         0.0956   
uniform   1         (0, 0, 0, 1, 1, 1, 2)     1.0       0.6502         0.0924   
                    (0, 1, 1, 1, 1, 2)        1.0       0.9998          0.183   
          2         (0, 0, 0, 1, 1, 1, 2)     1.0        0.707          0.088   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.1854   
          3         (0, 0, 0, 1, 1, 1, 2)  0.9996       0.8154         0.0974   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.1784   

                                                                         \
test                                        mann_whitney            reg   
unit                                      matching_group matching_group   
cov_type                                             NaN            NaN   
use_treatment_dummies                                  0              0   
err_type  err_scale ind_set                                               
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)         0.2072         0.8242   
                    (0, 1, 1, 1, 1, 2)             0.334         0.9796   
          2         (0, 0, 0, 1, 1, 1, 2)         0.1444         0.6126   
                    (0, 1, 1, 1, 1, 2)              0.23         0.9228   
          3         (0, 0, 0, 1, 1, 1, 2)         0.1178         0.4262   
                    (0, 1, 1, 1, 1, 2)            0.1636         0.7762   
uniform   1         (0, 0, 0, 1, 1, 1, 2)         0.2688          0.899   
                    (0, 1, 1, 1, 1, 2)            0.4298         0.9862   
          2         (0, 0, 0, 1, 1, 1, 2)         0.2468         0.8978   
                    (0, 1, 1, 1, 1, 2)            0.4146         0.9894   
          3         (0, 0, 0, 1, 1, 1, 2)         

In [7]:
num_cases = len(res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 1)] ) # num of DGPs

In [8]:
# comparison of cov_type: cluster vs HC3 (when unit == group, test = reg) == > more power with HC3
print( (res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 0)] <= res_pivot[('power_coeff1', 'reg', 'group', 'HC3', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff2', 'reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff1','reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)


1.0
0.9166666666666666
1.0


In [9]:
# comparison of unit: matching group vs group = > more power with group
print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'HC3', 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff2', 'reg', 'group', 'cluster', 1)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff2', 'reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'cluster', 1)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1', 'reg', 'matching_group', np.nan, 1)]  <= res_pivot[('power_coeff1', 'reg', 'group', 'HC3', 1)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1', 'mann_whitney', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff1', 'mann_whitney', 'group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2', 'mann_whitney', 'matching_group', np.nan, 0)]  <= res_pivot[('power_coeff2', 'mann_whitney', 'group', np.nan, 0)]).astype(int).sum()/num_cases)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.75


In [10]:
# comparison of test: regression (parametric) vs Mann-Whitney (nonparametric)

print( (res_pivot[('power_coeff1','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff1', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','reg', 'group', 'HC3', 1)] <= res_pivot[('power_coeff1', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases) 

print( (res_pivot[('power_coeff2','reg', 'group', 'cluster', 1)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','reg', 'group', 'HC3', 1)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','reg', 'group', 'cluster', 0)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','reg', 'group', 'HC3', 0)] <= res_pivot[('power_coeff2', 'mann_whitney','group', np.nan, 0)]).astype(int).sum()/num_cases)

print( (res_pivot[('power_coeff1','reg', 'matching_group', np.nan, 0)] <= res_pivot[('power_coeff2', 'mann_whitney','matching_group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff1','reg', 'matching_group', np.nan, 1)] <= res_pivot[('power_coeff1', 'mann_whitney','matching_group', np.nan, 0)]).astype(int).sum()/num_cases)
print( (res_pivot[('power_coeff2','reg', 'matching_group', np.nan , 1)] <= res_pivot[('power_coeff2', 'mann_whitney','matching_group', np.nan, 0)]).astype(int).sum()/num_cases)

0.6666666666666666
0.3333333333333333
0.5833333333333334
0.4166666666666667
0.5833333333333334
0.4166666666666667
0.25
1.0
0.4166666666666667


In [11]:
res_pivot

power_coeff1                  \
test                                               reg                   
unit                                             group                   
cov_type                                       cluster     HC3 cluster   
use_treatment_dummies                                1       1       0   
err_type  err_scale ind_set                                              
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)        0.446   0.742   0.985   
                    (0, 1, 1, 1, 1, 2)          0.7822  0.9544     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)       0.2102   0.597  0.8496   
                    (0, 1, 1, 1, 1, 2)          0.3566  0.7526  0.9962   
          3         (0, 0, 0, 1, 1, 1, 2)       0.1538   0.554  0.6264   
                    (0, 1, 1, 1, 1, 2)          0.2326  0.6438   0.933   
uniform   1         (0, 0, 0, 1, 1, 1, 2)       0.9918  0.9936  0.9992   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          2         (0, 0, 0, 1, 1, 1, 2)       0.9802  0.9872  0.9996   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   
          3         (0, 0, 0, 1, 1, 1, 2)       0.9742  0.9802  0.9996   
                    (0, 1, 1, 1, 1, 2)             1.0     1.0     1.0   

                                                                               \
test                                              mann_whitney            reg   
unit                                                     group matching_group   
cov_type                                      HC3          NaN            NaN   
use_treatment_dummies                           0            0              1   
err_type  err_scale ind_set                                                     
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)  0.9956         0.72          0.088   
                    (0, 1, 1, 1, 1, 2)        1.0       0.9778         0.1644   
          2         (0, 0, 0, 1, 1, 1, 2)  0.9534       0.5752         0.0656   
                    (0, 1, 1, 1, 1, 2)        1.0         0.77         0.1136   
          3         (0, 0, 0, 1, 1, 1, 2)  0.8748       0.4894         0.0646   
                    (0, 1, 1, 1, 1, 2)     0.9942       0.6138         0.0956   
uniform   1         (0, 0, 0, 1, 1, 1, 2)     1.0       0.6502         0.0924   
                    (0, 1, 1, 1, 1, 2)        1.0       0.9998          0.183   
          2         (0, 0, 0, 1, 1, 1, 2)     1.0        0.707          0.088   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.1854   
          3         (0, 0, 0, 1, 1, 1, 2)  0.9996       0.8154         0.0974   
                    (0, 1, 1, 1, 1, 2)        1.0          1.0         0.1784   

                                                                         \
test                                        mann_whitney            reg   
unit                                      matching_group matching_group   
cov_type                                             NaN            NaN   
use_treatment_dummies                                  0              0   
err_type  err_scale ind_set                                               
norm_corr 1         (0, 0, 0, 1, 1, 1, 2)         0.2072         0.8242   
                    (0, 1, 1, 1, 1, 2)             0.334         0.9796   
          2         (0, 0, 0, 1, 1, 1, 2)         0.1444         0.6126   
                    (0, 1, 1, 1, 1, 2)              0.23         0.9228   
          3         (0, 0, 0, 1, 1, 1, 2)         0.1178         0.4262   
                    (0, 1, 1, 1, 1, 2)            0.1636         0.7762   
uniform   1         (0, 0, 0, 1, 1, 1, 2)         0.2688          0.899   
                    (0, 1, 1, 1, 1, 2)            0.4298         0.9862   
          2         (0, 0, 0, 1, 1, 1, 2)         0.2468         0.8978   
                    (0, 1, 1, 1, 1, 2)            0.4146         0.9894   
          3         (0, 0, 0, 1, 1, 1, 2)         

In [12]:
# share of DGPs in grid with power >= 0.8, 0.9 
print(((res_pivot >= 0.8).astype(int).sum().sum())/(res_pivot == res_pivot).astype(int).sum().sum())
print(((res_pivot >= 0.9).astype(int).sum().sum())/(res_pivot == res_pivot).astype(int).sum().sum())

0.6410256410256411
0.5128205128205128


Test rejection rate when null is true/ type I error

In [13]:
# test rejection rate when null is true
ind_set = [[0]]

# simulation
res_null = pd.DataFrame()

combinations = list(product(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, err_type, err_scale, ind_set, params))
for comb in combinations: 
    #print(comb)
    
    ps = power_simulation(   
            num_runs = comb[0], 
            sig_level = comb[1], 
            n_matching_grp_per_treatment = comb[2], 
            n_obs_per_matching_grp = comb[3], 
            small_coal_worth = comb[4], 
            unit = params[comb[8]]["unit"], 
            err_type = comb[5], 
            err_scale = comb[6],
            ind_set = comb[7], 
            test = params[comb[8]]["test"],
            use_treatment_dummies = params[comb[8]]["use_treatment_dummies"], 
            cov_type = params[comb[8]]["cov_type"]
        )
    r = pd.DataFrame.from_dict(ps, orient = "index").transpose()
    res_null = pd.concat([res_null, r], axis = 0)

In [14]:
res_null["small_coal_worth"] = res_null["small_coal_worth"].apply(lambda x: tuple(x))
res_null["ind_set"] = res_null["ind_set"].apply(lambda x: tuple(x))

res_null_pivot = res_null.pivot(index = ["err_type", "err_scale", "ind_set"], columns = ["test", "unit", "cov_type", "use_treatment_dummies"], values = ["power_coeff1", "power_coeff2"])
res_null_pivot

power_coeff1                                       \
test                                 reg                         mann_whitney   
unit                               group                                group   
cov_type                         cluster     HC3 cluster     HC3          NaN   
use_treatment_dummies                  1       1       0       0            0   
err_type  err_scale ind_set                                                     
norm_corr 1         (0,)          0.1072  0.5468  0.0946  0.5444       0.2994   
          2         (0,)          0.1116  0.5506  0.0944   0.543        0.302   
          3         (0,)          0.1002  0.5514  0.0884  0.5258       0.2834   
uniform   1         (0,)           0.108   0.053  0.0942  0.0478       0.0454   
          2         (0,)          0.1094  0.0478  0.0938  0.0574       0.0514   
          3         (0,)          0.1002  0.0472  0.1022  0.0504       0.0478   

                                                                          \
test                                   reg   mann_whitney            reg   
unit                        matching_group matching_group matching_group   
cov_type                               NaN            NaN            NaN   
use_treatment_dummies                    1              0              0   
err_type  err_scale ind_set                                                
norm_corr 1         (0,)            0.0526         0.0406         0.0534   
          2         (0,)            0.0506         0.0478          0.047   
          3         (0,)            0.0472         0.0478         0.0462   
uniform   1         (0,)            0.0492         0.0386         0.0504   
          2         (0,)            0.0492         0.0396          0.049   
          3         (0,)            0.0514         0.0412         0.0492   

                            power_coeff2                                    \
test                                 reg                      mann_whitney   
unit                               group                             group   
cov_type                         cluster     HC3 cluster  HC3          NaN   
use_treatment_dummies                  1       1       0    0            0   
err_type  err_scale ind_set                                                  
norm_corr 1         (0,)          0.1062   0.549     NaN  NaN       0.3032   
          2         (0,)          0.1018  0.5534     NaN  NaN       0.2964   
          3         (0,)          0.0988   0.551     NaN  NaN       0.2916   
uniform   1         (0,)          0.1042  0.0522     NaN  NaN       0.0454   
          2         (0,)          0.1046  0.0518     NaN  NaN       0.0508   
          3         (0,)          0.1048  0.0498     NaN  NaN        0.045   

                                                                          
test                                   reg   mann_whitney            reg  
unit                        matching_group matching_group matching_group  
cov_type                               NaN            NaN            NaN  
use_treatment_dummies                    1              0              0  
err_type  err_scale ind_set                                               
norm_corr 1         (0,)            0.0468         0.0496            NaN  
          2         (0,)            0.0446         0.0416            NaN  
          3         (0,)            0.0488         0.0522            NaN  
uniform   1         (0,)             0.052         0.0398            NaN  
          2         (0,)             0.048         0.0398            NaN  
          3         (0,)            0.0528         0.0398            NaN